In [5]:
movie_idx = np.array([[0, 3, 4, 5, 6, 7, 8],
       [0, 3, 5, 6, 7, 8, 9],
       [0, 3, 4, 5, 6, 7, 8],
       [0, 3, 4, 5, 6, 7, 8]])

In [47]:
movie_idx

array([[0, 3, 4, 5, 6, 7, 8],
       [0, 3, 5, 6, 7, 8, 9],
       [0, 3, 4, 5, 6, 7, 8],
       [0, 3, 4, 5, 6, 7, 8]])

In [135]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np
import random

movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')





In [136]:
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))


In [138]:
movie_idx[0]

array([0, 3, 4, 5, 6, 7, 8])

In [114]:
movies[movies.movie_id == '0']

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [87]:
0 in movie_idx

True

In [124]:
movie_list = []
for i in movie_idx[:]:
    print(i)
    print(movies[movies.movie_id == str(i)])

[0 3 4 5 6 7 8]
Empty DataFrame
Columns: [movie_id, title, release_date, video_release_date, imdb_url, genre_unknown, Action, Adventure, Animation, Children, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
Index: []

[0 rows x 24 columns]
[0 3 5 6 7 8 9]
Empty DataFrame
Columns: [movie_id, title, release_date, video_release_date, imdb_url, genre_unknown, Action, Adventure, Animation, Children, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
Index: []

[0 rows x 24 columns]
[0 3 4 5 6 7 8]
Empty DataFrame
Columns: [movie_id, title, release_date, video_release_date, imdb_url, genre_unknown, Action, Adventure, Animation, Children, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western]
Index: []

[0 rows x 24 columns]
[0 3 4 5 6 7 8]
Empty DataFrame
Columns: [movie_id, title, re

In [122]:
movie_list

[]

In [21]:
from urllib.request import urlretrieve
import zipfile
import pandas as pd
import numpy as np
import random

users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'data/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'data/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'data/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))


genre_occurences = movies[genre_cols].sum().to_dict()

genres_encoded = {x: i for i, x in enumerate(genre_cols)}



def get_genres(movies, genres):
    def get_all_genres(gs):
        active = [str(genres_encoded[genre]) for genre, g in zip(genres, gs) if g==1]
        if len(active) == 0:
            return '0'
        return ','.join((active))
    movies['all_genres'] = [
        get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

get_genres(movies, genre_cols) # 각 유저가 본 장르 얻기

rating_details_sample = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

rating_details_sample['user_id']=rating_details_sample['user_id'].astype(int)
rating_details_sample['movie_id']=rating_details_sample['movie_id'].astype(int)
rating_details_sample=rating_details_sample.set_index(['user_id','unix_timestamp']).sort_index()
rating_details_sample =rating_details_sample.reset_index()

rating_details_sample['movie_type']=np.where(rating_details_sample['rating'] >= 3, 'like', 'dislike') # 3보다 크면 like
rating_details_sample['movie_name']=rating_details_sample['title'].str[:-6] # 년도 부분 자르기

user_ids = rating_details_sample["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

movie_ids = rating_details_sample["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}


title_ids = rating_details_sample["movie_name"].unique().tolist()
title2title_encoded = {x: i for i, x in enumerate(title_ids)}
title_encoded2title = {i: x for i, x in enumerate(title_ids)}

rating_details_sample["user"] = rating_details_sample["user_id"].map(user2user_encoded)
rating_details_sample["movie"] = rating_details_sample["movie_id"].map(movie2movie_encoded)
rating_details_sample["title_d"] = rating_details_sample["movie_name"].map(title2title_encoded)
# print(rating_details_sample["movie"] )
sample_data=rating_details_sample[['user','occupation','sex']]
sample_data=sample_data.reset_index()

movie_list = rating_details_sample.groupby(['user','movie_type'])['movie'].apply(list).reset_index()
title_list = rating_details_sample.groupby(['user'])['title_d'].apply(list).reset_index()
genre_list = rating_details_sample.groupby(['user'])['all_genres'].unique().apply(list).reset_index()

print(movie_list)

# Get the unique set of genre for all the users
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x: list(set(','.join(x))) ) # 중복제거
genre_list['all_genres']=genre_list['all_genres'].apply(lambda x:[ x for x in x if x.isdigit() ])

user_video_list = movie_list.pivot(index='user', columns='movie_type', values='movie').reset_index()
user_video_list.fillna(rating_details_sample["movie"].max()+1, inplace=True)

sample_data = sample_data.drop('index',axis=1)
sample_data = sample_data.drop_duplicates()

user_final_list = pd.merge(user_video_list,title_list, how= 'left')
user_title_list1 = pd.merge(user_final_list,genre_list, how='left')
user_title_list = pd.merge(user_title_list1,sample_data, how='left')

user_title_list['like'] =user_title_list['like'].apply(lambda x: x if type(x) is list else [x])
user_title_list['dislike'] =user_title_list['dislike'].apply(lambda x: x if type(x) is list else [x])
user_title_list['predict_labels'] = user_title_list['like'].apply(lambda x: int(random.uniform(0,rating_details_sample["movie"].max()))) #label을 마지막 값으로..
# user_title_list['predict_labels'] = user_title_list['like'].apply(lambda x: (x[-1])) #label을 마지막 값으로..
# user_title_list['like']=user_title_list['like'].apply(lambda x: (x[:-1])) # 마지막 인덱스 레이블로 쓰고 제외
# user_title_list['like']=user_title_list['like'].apply(lambda x: [rating_details_sample["movie"].max()+2] if x == [] else x)

# user_title_list['predict_labels'] = user_title_list['predict_labels'].apply(lambda x: 1 if x == 10 else x)

user_title_list_e=user_title_list[(user_title_list.user >= 1)&
                                  (user_title_list.user <= 5)]
test_data=user_title_list[(user_title_list.user >= 6)&
                                  (user_title_list.user <= 9)]



0     0
1     1
2     2
3     3
4     4
5     5
6     0
7     6
8     6
9     7
10    8
11    5
12    6
13    2
14    9
15    4
16    9
17    9
Name: movie, dtype: int64
    user movie_type      movie
0      0    dislike     [0, 1]
1      0       like     [2, 3]
2      1       like     [4, 5]
3      2       like     [0, 6]
4      3    dislike        [6]
5      4    dislike        [7]
6      5       like        [8]
7      6       like     [5, 6]
8      7       like  [2, 9, 4]
9      8    dislike        [9]
10     9    dislike        [9]


In [17]:
user_title_list_e


,user,dislike,like,title_d,all_genres,occupation,sex,predict_labels
1,1,[10],"[4, 5]","[4, 5]","[5, 1, 8, 2, 6]",other,F,5
2,2,[10],"[0, 6]","[0, 6]","[5, 4, 1, 8, 6]",writer,M,3
3,3,[6],[10],[6],"[8, 4, 5]",technician,M,2
4,4,[7],[10],[7],"[8, 1, 5]",other,F,8
5,5,[10],[8],[8],[8],executive,M,3


In [28]:
a = pd.Series(movie_idx, index=range(6,10) ,columns=['test1','test3','test3','test4','test5','test6','test7'])
# a['cc']

TypeError: __init__() got an unexpected keyword argument 'columns'